Dataset Link: http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset

## Step 0: Importing Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import cv2 as cv2

## Step 1: Importing Data

In [6]:
df_train = pd.read_csv('Train.csv')
df_test = pd.read_csv('Test.csv')

df_train["is_train"] = 1
df_test["is_train"] = 0

df = pd.concat([df_train, df_test])

In [7]:
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51839 entries, 0 to 51838
Data columns (total 9 columns):
Width       51839 non-null int64
Height      51839 non-null int64
Roi.X1      51839 non-null int64
Roi.Y1      51839 non-null int64
Roi.X2      51839 non-null int64
Roi.Y2      51839 non-null int64
ClassId     51839 non-null int64
Path        51839 non-null object
is_train    51839 non-null int64
dtypes: int64(8), object(1)
memory usage: 3.6+ MB


In [8]:
df.head()

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path,is_train
0,27,26,5,5,22,20,20,Train/20/00020_00000_00000.png,1
1,28,27,5,6,23,22,20,Train/20/00020_00000_00001.png,1
2,29,26,6,5,24,21,20,Train/20/00020_00000_00002.png,1
3,28,27,5,6,23,22,20,Train/20/00020_00000_00003.png,1
4,28,26,5,5,23,21,20,Train/20/00020_00000_00004.png,1


In [9]:
df.ClassId.nunique(), df.ClassId.unique()

(43, array([20,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42], dtype=int64))

## Step 3: Modelling Data

In [1]:
from tensorflow import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Activation, Conv2D, Flatten, Lambda
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.optimizers import Adam

In [11]:
raw = df[['Path', 'is_train', 'ClassId']]

In [12]:
raw = pd.get_dummies(raw, columns=['ClassId'])

In [31]:
raw_train = raw[(raw.is_train == 1)]
raw_test = raw[(raw.is_train == 0)]
X = raw_train.Path
y = raw_train.iloc[:,2:]

In [32]:

from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.1, shuffle=True)

In [33]:
y_valid.shape, y_train.shape

((3921, 43), (35288, 43))

In [59]:
def generator(samples, batch_size=32, is_test= None):
    num_samples = len(samples)
   
    while 1: 
        shuffle(samples) #shuffling the total images
        for offset in range(0, num_samples, batch_size):
            
            batch_samples = samples[offset:offset+batch_size]

            images = []
            labels = []
            for batch_sample in batch_samples:
#                 print(batch_sample[0])
#                 print("-----------------------")
                local_image =  cv2.cvtColor( cv2.imread(batch_sample[0]) , cv2.COLOR_BGR2RGB )
                local_image = cv2.resize(local_image, (50,50))
                images.append(local_image)
                labels.append(batch_sample[1:])
            
            X_batch = np.array(images)
            y_batch = np.array(labels)
            
            if(is_test):
                yield shuffle(X_batch)
            else:
                yield shuffle(X_batch, y_batch) 

train_samples = pd.concat([X_train, y_train], axis=1)
valid_samples = pd.concat([X_valid, y_valid], axis=1)


train_samples = np.array(train_samples)
valid_samples = np.array(valid_samples)
batch_size = 4

train_generator = generator(train_samples, batch_size)
valid_generator = generator(valid_samples, batch_size)

In [67]:
raw_test_filtered = np.array( raw_test.drop(columns=['is_train']) )

In [35]:
model = Sequential()
model.add(Lambda(lambda x: x/255 - 0.5,  input_shape=(50,50,3)))
model.add(Conv2D(3,(3,3),  activation='relu'))
model.add(Conv2D(8,(3,3),strides=(2,2), activation='relu'))
model.add(Conv2D(32,(5,5), strides=(2,2), activation='relu'))
model.add(Conv2D(64,(5,5), strides=(2,2), activation='relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(43))
model.add(Activation('softmax'))

In [36]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 50, 50, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 3)         84        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 23, 23, 8)         224       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 32)        6432      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 3, 64)          51264     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               73856     
__________

In [37]:
model.compile(loss = 'categorical_crossentropy', optimizer= Adam(lr=0.0001), metrics=['accuracy'] )

In [38]:
model.fit_generator(train_generator, epochs=5,  verbose = 1, steps_per_epoch=len(train_samples)//batch_size ,  validation_data = valid_generator, validation_steps = len(valid_samples)//batch_size )

Epoch 1/5
8822/8822 [==============================] - 774s 88ms/step - loss: 1.4044 - acc: 0.5968 - val_loss: 0.5718 - val_acc: 0.8370
Epoch 2/5
8822/8822 [==============================] - 179s 20ms/step - loss: 0.3836 - acc: 0.8903 - val_loss: 0.2729 - val_acc: 0.9260
Epoch 3/5
8822/8822 [==============================] - 451s 51ms/step - loss: 0.2055 - acc: 0.9434 - val_loss: 0.1625 - val_acc: 0.9581
Epoch 4/5
8822/8822 [==============================] - 454s 51ms/step - loss: 0.1261 - acc: 0.9653 - val_loss: 0.1474 - val_acc: 0.9602
Epoch 5/5
8822/8822 [==============================] - 463s 52ms/step - loss: 0.0826 - acc: 0.9775 - val_loss: 0.1152 - val_acc: 0.9676


## Step 4: Saving and Evaluating

In [39]:
model.save('model.h5')

In [101]:

model.evaluate_generator( generator(raw_test_filtered, batch_size), steps=len(raw_test_filtered)/batch_size)

[0.5314731922336938, 0.8850356294536817]